# Week 2 Exercise Solution

## Technical Question Assistant with Gradio UI, Streaming, Model Switching, and System Prompt

This notebook demonstrates a tool that takes a technical question and responds with an explanation using both OpenAI (gpt-4o-mini) and Ollama (llama3:latest) models, with a Gradio UI for interaction.

---

## Step 1: Import Required Libraries

We need the following libraries:
- `os`, `dotenv` for environment variables
- `openai` for OpenAI API
- `ollama` for local LLM
- `gradio` for UI

> If not installed, run `pip install openai python-dotenv ollama gradio`

In [1]:
# Import required libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
import ollama
import gradio as gr

In [2]:
load_dotenv(override=True)
openrouter_api_key = os.getenv("OPEN_ROUTER_API_KEY")

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3:latest'

SYSTEM_PROMPT = "You are an expert technical explainer. Provide clear, concise, and accurate answers to technical questions."

---

## Step 2: Streaming Answer Functions

These functions stream answers from OpenAI and Ollama models.

In [10]:
# Streaming OpenAI response
def stream_openai_response(question, system_prompt=SYSTEM_PROMPT):
    if not openrouter_api_key:
        return "No OpenRouter API key found."
    openai = OpenAI(api_key=openrouter_api_key, base_url="https://openrouter.ai/api/v1")
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": question}],
        stream=True
    )
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [11]:
# Streaming Ollama response
def stream_ollama_response(question, system_prompt=SYSTEM_PROMPT):
    stream = ollama.chat(model=MODEL_LLAMA, messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": question}], stream=True)
    response = ""
    for chunk in stream:
        if 'message' in chunk and 'content' in chunk['message']:
            response += chunk['message']['content']
            yield response

---

## Step 3: Gradio UI

This cell creates a Gradio UI for entering questions, selecting models, and streaming answers.

In [13]:
# Gradio UI function

def answer_question(question, model):
    if model == "OpenAI (gpt-4o-mini)":
        for partial in stream_openai_response(question):
            yield partial
    else:
        for partial in stream_ollama_response(question):
            yield partial

with gr.Blocks() as demo:
    gr.Markdown("""
    # Week 2 Technical Question Answerer
    Enter your technical question below, select a model, and get a streaming expert answer!
    """)
    with gr.Row():
        question = gr.Textbox(label="Technical Question", placeholder="Ask your technical question here...")
        model = gr.Radio(["OpenAI (gpt-4o-mini)", "Ollama (llama3:latest)"], value="OpenAI (gpt-4o-mini)", label="Model")
    answer = gr.Textbox(label="Streaming Answer", lines=8)
    submit = gr.Button("Get Answer")

    submit.click(answer_question, inputs=[question, model], outputs=answer, queue=True)

    gr.Markdown("""
    *Bonus: Try adding audio input/output or tool use!*
    """)

demo.launch()


* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.
